In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import nltk
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
#title
asins = open('../CategoryCluster/C10/UK/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
# ASINs = ASINs[:-1]
len(ASINs)

7640

In [3]:
#synopses
review = open('../CategoryCluster/C10/UK/pre/review.csv').read().split('\n')
review[0]
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
len(Reviews)


7640

In [4]:
#ranks
ranks = open('../CategoryCluster/C10/UK/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split(',')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

7640

In [5]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [6]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
# 这里我定义了一个分词器（tokenizer）和词干分析器（stemmer），它们会输出给定文本词干化后的词集合
 
def tokenize_and_stem(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
 
def tokenize_only(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [8]:
# 非常不 pythonic，一点也不！
# 扩充列表后变成了非常庞大的二维（flat）词汇表
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews:
    allwords_stemmed = tokenize_and_stem(i) #对每个评价进行分词和词干化
    totalvocab_stemmed.extend(allwords_stemmed) # 扩充“totalvocab_stemmed”列表
 
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

## Tf-idf and document similarity

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# 定义向量化参数
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer(max_df=0.98, max_features=200000,
                                 min_df=0.02, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
%time tf =  vectorizer.fit_transform(Reviews)
%time tfidf = transformer.fit_transform(tf) # 向量化
 

CPU times: user 6.95 s, sys: 94.5 ms, total: 7.05 s
Wall time: 7.27 s
CPU times: user 2.08 ms, sys: 357 µs, total: 2.44 ms
Wall time: 2.3 ms


In [10]:
#获取词袋模型中的前10个词语 
word = vectorizer.get_feature_names()
print(word[:10])

["'s", 'absolut', 'ador', 'amaz', 'ani', 'arriv', 'babi', 'big', 'birthday', 'bit']


In [11]:
# 词袋模型中共有几个词语
len(word)

105

In [12]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在第i类评论中的tf-idf权重  
weight = tfidf.toarray()  
weight

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.27828497,  0.29361554],
       ..., 
       [ 0.31325465,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60004637,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
weight.shape

(7640, 105)

In [14]:
#词频矩阵
tfidframe = pd.DataFrame(np.round(weight, 2))

In [15]:
tfidframe[:10]

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
1,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.30,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
2,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.22,0.0,0.00,0.0,0.00,0.0,0.0,0.28,0.29
3,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
4,0.0,0.0,0.0,0.0,0.53,0.00,0.0,0.0,0.0,0.0,...,0.0,0.34,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
5,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
6,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.35,0.0,0.00,0.0,0.0,0.00,0.00
7,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
8,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.26,0.0,0.46,0.0,0.0,0.00,0.00
9,0.0,0.0,0.0,0.0,0.00,0.55,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00


## cosine similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf)
dist = 1 - similarity

In [17]:
print(dist)

[[ -2.22044605e-16   1.00000000e+00   6.84754295e-01 ...,   8.32061968e-01
    9.48478351e-01   8.45575011e-01]
 [  1.00000000e+00   0.00000000e+00   9.33648233e-01 ...,   8.56515977e-01
    8.61784014e-01   6.89722658e-01]
 [  6.84754295e-01   9.33648233e-01  -2.22044605e-16 ...,   8.11144872e-01
    8.03198079e-01   8.33989995e-01]
 ..., 
 [  8.32061968e-01   8.56515977e-01   8.11144872e-01 ...,   0.00000000e+00
    2.51704483e-01   5.18331101e-01]
 [  9.48478351e-01   8.61784014e-01   8.03198079e-01 ...,   2.51704483e-01
   -2.22044605e-16   5.64963330e-01]
 [  8.45575011e-01   6.89722658e-01   8.33989995e-01 ...,   5.18331101e-01
    5.64963330e-01  -2.22044605e-16]]


In [18]:
similarity_df = pd.DataFrame(similarity)
similarity_df[:10]

,0,1,2,3,4,5,6,7,8,9,...,7630,7631,7632,7633,7634,7635,7636,7637,7638,7639
0,1.000000,0.000000,0.315246,0.252964,0.148946,0.000000,0.086754,0.274034,0.00000,0.000000,...,0.204332,0.039199,0.020965,0.122771,0.032713,0.149790,0.000000,0.167938,0.051522,0.154425
1,0.000000,1.000000,0.066352,0.000000,0.102843,0.256754,0.102597,0.000000,0.00000,0.373741,...,0.167969,0.106872,0.146901,0.245437,0.172469,0.183057,0.146491,0.143484,0.138216,0.310277
2,0.315246,0.066352,1.000000,0.092127,0.188785,0.076113,0.297893,0.000000,0.07035,0.000000,...,0.267288,0.293487,0.143060,0.231424,0.167167,0.291850,0.150084,0.188855,0.196802,0.166010
3,0.252964,0.000000,0.092127,1.000000,0.000000,0.131486,0.058049,0.573697,0.12153,0.000000,...,0.033317,0.026229,0.053710,0.097904,0.145723,0.057299,0.000000,0.044912,0.025542,0.083645
4,0.148946,0.102843,0.188785,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.399614,0.118227,0.174922,0.115261,0.235134,0.129138,0.224838,0.101221,0.130348,0.341532
5,0.000000,0.256754,0.076113,0.131486,0.000000,1.000000,0.000000,0.000000,0.23915,0.190354,...,0.027526,0.000000,0.070051,0.387503,0.146827,0.000000,0.000000,0.136907,0.180424,0.050551
6,0.086754,0.102597,0.297893,0.058049,0.000000,0.000000,1.000000,0.062884,0.09271,0.359971,...,0.164326,0.027062,0.099191,0.123317,0.071798,0.059118,0.090135,0.046338,0.164624,0.144016
7,0.274034,0.000000,0.000000,0.573697,0.000000,0.000000,0.062884,1.000000,0.00000,0.000000,...,0.000000,0.191307,0.015197,0.027701,0.023712,0.062072,0.000000,0.048653,0.062668,0.208787
8,0.000000,0.000000,0.070350,0.121530,0.000000,0.239150,0.092710,0.000000,1.00000,0.000000,...,0.188630,0.000000,0.030301,0.138348,0.094562,0.000000,0.000000,0.000000,0.058532,0.000000
9,0.000000,0.373741,0.000000,0.000000,0.000000,0.190354,0.359971,0.000000,0.00000,1.000000,...,0.073039,0.000000,0.000000,0.104718,0.000000,0.000000,0.000000,0.000000,0.045953,0.117998


## K-means clustering

In [19]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 1.01 s, sys: 67.8 ms, total: 1.08 s
Wall time: 990 ms
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [20]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [21]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters}
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster'])

In [22]:
# frame[:5]

In [23]:
# frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

In [24]:
# km.inertia_

In [25]:
# grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
# grouped.mean() # 每个聚类的平均排名（0 到 5）

In [26]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
# for i in range(num_clusters):
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() # 空行
#     print() # 空行

In [27]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# # print(order_centroids)
# for i in range(num_clusters):
#     print("Cluster %d:" % i, end='')
#     time.sleep(1)
#     for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
#         print(' %s' % word[ind]  , end=',')
#     print() # 空行
#     print() # 空行

## LDA

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

In [29]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [30]:
%time lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 54.4 s, sys: 595 ms, total: 55 s
Wall time: 57.2 s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
%time lda_matrix = lda.fit_transform(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 52 s, sys: 232 ms, total: 52.2 s
Wall time: 53.1 s


In [32]:
lda_matrix #表示每一個文檔屬於每一個聚類的概率

array([[ 0.01111374,  0.22230756,  0.0111145 , ...,  0.0111127 ,
         0.48173756,  0.21816941],
       [ 0.01428817,  0.01428854,  0.18991668, ...,  0.01428601,
         0.21199854,  0.014294  ],
       [ 0.00714289,  0.47154482,  0.0071437 , ...,  0.25746871,
         0.00714367,  0.13417729],
       ..., 
       [ 0.005264  ,  0.67213262,  0.00526354, ...,  0.00526483,
         0.00526423,  0.00526316],
       [ 0.00075194,  0.41997452,  0.00075201, ...,  0.00075202,
         0.02148779,  0.00075209],
       [ 0.00526316,  0.40907557,  0.00526454, ...,  0.00526362,
         0.37408554,  0.00526595]])

In [33]:
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [34]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.011114,0.222308,0.011115,0.011111,0.011111,0.011111,0.011111,0.011113,0.481738,0.218169
1,0.014288,0.014289,0.189917,0.498070,0.014287,0.014286,0.014286,0.014286,0.211999,0.014294
2,0.007143,0.471545,0.007144,0.093949,0.007144,0.007143,0.007143,0.257469,0.007144,0.134177
3,0.016670,0.016669,0.016667,0.016667,0.016667,0.349995,0.016668,0.016669,0.516661,0.016668
4,0.016668,0.016671,0.016667,0.016667,0.016673,0.235550,0.212015,0.016670,0.231816,0.220603


In [35]:
%time km_lda =km.fit_transform(features)

CPU times: user 278 ms, sys: 15.4 ms, total: 293 ms
Wall time: 308 ms


In [36]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [37]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews})
ldaframe = ldaframe[['title','rank','review']]
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)
ldaClusterframe[:5]

,title,rank,review,ClusterLabel
0,B000284ZNI_1,5.0,"Our Granddaughter who is 14 months loves it , ...",0
1,B000284ZNI_2,3.0,"Toy itself looks ok, but as mentioned by many ...",8
2,B000284ZNI_3,5.0,I had this toy for my two little ones and have...,2
3,B000284ZNI_4,5.0,Bought for Xmas present for my granddaughter s...,0
4,B000284ZNI_5,5.0,Briliant child/infant toy. Its squashy and ha...,1


In [38]:
ldaClusterframe['ClusterLabel'].value_counts()

1    1285
2     822
6     818
5     815
0     801
8     776
9     652
3     605
4     605
7     461
Name: ClusterLabel, dtype: int64

In [39]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    4.672909
1    2.617899
2    4.575426
3    4.089256
4    4.732231
5    4.387730
6    4.298289
7    4.555315
8    3.675258
9    4.656442
Name: rank, dtype: float64

In [40]:
ldaClusterframe['rank'].describe()

count    7640.000000
mean        4.094895
std         1.517373
min         0.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: rank, dtype: float64

In [41]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,review,ClusterLabel
0,B000284ZNI_1,5.0,"Our Granddaughter who is 14 months loves it , ...",0
1374,B01MFCMO1M_5,5.0,My granddaughter loves this doll. Trolls is he...,0
4950,B018S366RQ_1,5.0,My niece absolutely loves Teletubbies so she a...,0
4958,B018S366RQ_9,5.0,Classic toy and my Grandaught loves it.,0
1351,B00RY3C6MQ_2,4.0,This was purchased as a gift. They are a litt...,0
4982,B071CV6KWH_3,2.0,My granddaughter loves it but it has already s...,0
5010,B074J57L4L_1,5.0,My granddaughter loves this toy. It is big an...,0
5021,B000WNZ1SQ_2,4.0,good soft stuffing perfect for toy knitting,0
1337,B00XSPEW9O_8,5.0,my son loves him,0
1336,B00XSPEW9O_7,5.0,Granddaughter loves it,0


In [42]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
#     print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        
        
    print() # 空行
    print() # 空行

 love, daughter, daughter love, deliveri, look, children, packag, teddi, s, play, bit, t, excel, littl, ani, bought, product, use, item, arriv, came, say, friend, thing, nice, think, birthday, day, buy, absolut,

 love, littl, toy, babi, kid, great, girl, play, month, absolut, bought, sound, make, colour, use, old, think, ador, buy, s, fun, recommend, like, realli, product, onli, quit, price, gift, lot,

 squishi, smell, realli, rise, nice, slow, came, slow rise, veri, super, like, recommend, soft, amaz, packag, definit, good, fun, bit, expect, day, 's, price, colour, size, bought, play, come, great, love,

 like, just, n't, doe, look, time, arriv, realli, onli, pictur, doe n't, 's, good, t, thing, say, s, money, got, play, want, come, bit, order, sound, expect, item, disappoint, use, worth,

 's, cute, veri, littl, small, friend, soft, veri cute, size, bit, look, come, big, say, ani, like, perfect, teddi, just, birthday, want, quit, love, child, got, son, niec, think, nice, make,

 ch

In [43]:
order_centroids

array([[ 51,  20,  21, ...,  82,  27,  39],
       [ 51,  48,  96, ...,  91,  75,  80],
       [ 87,  83,  73, ...,  99,   6,  40],
       ..., 
       [ 51,  58, 103, ..., 100,  75,  80],
       [ 51,  96,  39, ...,  79,  80,  75],
       [ 35,  70,  98, ...,  28,  75,  80]])

In [44]:
ftf = transformer.fit_transform(order_centroids)
ftf = ftf.toarray()

In [45]:
#词频矩阵
tfidMMframe = pd.DataFrame(np.round(ftf, 2))
tfidMMframe[:5]

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,0.09,0.03,0.03,0.04,0.08,0.02,0.10,0.14,0.12,0.10,...,0.17,0.17,0.13,0.07,0.06,0.05,0.12,0.13,0.04,0.06
1,0.09,0.08,0.15,0.01,0.07,0.07,0.05,0.10,0.09,0.00,...,0.14,0.16,0.13,0.17,0.16,0.03,0.06,0.15,0.12,0.13
2,0.15,0.13,0.12,0.12,0.09,0.13,0.02,0.13,0.16,0.14,...,0.14,0.17,0.14,0.06,0.03,0.06,0.17,0.16,0.01,0.06
3,0.08,0.07,0.09,0.04,0.08,0.15,0.01,0.12,0.10,0.10,...,0.05,0.17,0.12,0.13,0.13,0.03,0.16,0.16,0.17,0.06
4,0.00,0.03,0.16,0.08,0.13,0.05,0.13,0.16,0.13,0.01,...,0.17,0.07,0.17,0.03,0.15,0.16,0.13,0.13,0.12,0.06


In [46]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        print(' %f' % lda.components_[i][ind-1], end=',')
        print(' %f' % (tfidMMframe[i:i+1][ind]), end=',')#a词在所属群中出现的频率
        print(' %f' % max(tfidframe[:][ind]), end=',') #a词与所有顾客评论中出现最高词频的相对指数
        print(' %f' % (tfidMMframe[i:i+1][ind]/max(tfidframe[:][ind])), end='\n')
        
    print('') # 空行
    print() # 空行

Cluster 0:
 love, 0.100013, 0.000000, 1.000000, 0.000000
 daughter, 0.100008, 0.020000, 1.000000, 0.020000
 daughter love, 1123.789727, 0.140000, 0.720000, 0.194444
 deliveri, 0.100005, 0.090000, 0.910000, 0.098901
 look, 11.443411, 0.120000, 0.880000, 0.136364
 children, 0.100009, 0.010000, 0.920000, 0.010870
 packag, 0.100016, 0.020000, 1.000000, 0.020000
 teddi, 14.635696, 0.100000, 1.000000, 0.100000
 s, 0.100002, 0.050000, 1.000000, 0.050000
 play, 0.100003, 0.020000, 1.000000, 0.020000
 bit, 0.100018, 0.100000, 1.000000, 0.100000
 t, 0.100004, 0.090000, 0.850000, 0.105882
 excel, 0.100003, 0.000000, 1.000000, 0.000000
 littl, 0.100014, 0.060000, 1.000000, 0.060000
 ani, 0.100004, 0.080000, 0.690000, 0.115942
 bought, 22.526277, 0.010000, 1.000000, 0.010000
 product, 0.100009, 0.140000, 1.000000, 0.140000
 use, 0.100012, 0.130000, 0.780000, 0.166667
 item, 0.100009, 0.150000, 1.000000, 0.150000
 arriv, 9.964675, 0.020000, 0.890000, 0.022472
 came, 0.100017, 0.050000, 1.000000, 0.0

 great, 0.100006, 0.020000, 1.000000, 0.020000
 happi, 869.547729, 0.040000, 1.000000, 0.040000
 veri, 0.100016, 0.130000, 1.000000, 0.130000
 product, 0.100023, 0.070000, 1.000000, 0.070000
 gift, 0.100015, 0.100000, 1.000000, 0.100000
 thank, 25.013868, 0.170000, 1.000000, 0.170000
 veri happi, 0.100005, 0.130000, 0.710000, 0.183099
 bought, 0.100015, 0.040000, 1.000000, 0.040000
 buy, 144.160164, 0.120000, 0.910000, 0.131868
 niec, 0.100014, 0.120000, 0.920000, 0.130435
 deliveri, 0.100019, 0.160000, 0.910000, 0.175824
 purchas, 359.626035, 0.090000, 0.820000, 0.109756
 absolut, 0.100009, 0.070000, 1.000000, 0.070000
 son, 0.100014, 0.090000, 0.900000, 0.100000
 ador, 38.354523, 0.160000, 1.000000, 0.160000
 present, 0.100013, 0.090000, 0.900000, 0.100000
 teddi, 0.100007, 0.130000, 1.000000, 0.130000
 love, 0.100017, 0.100000, 1.000000, 0.100000
 packag, 0.100019, 0.080000, 1.000000, 0.080000
 recommend, 0.100023, 0.040000, 1.000000, 0.040000
 arriv, 0.100020, 0.150000, 0.890000, 0

In [47]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      13.238755        1       1  0.198731 -0.086126
4      12.740478        1       2 -0.115794  0.046473
9      11.612849        1       3 -0.169212 -0.180266
2      11.274390        1       4 -0.113255 -0.075087
3      11.257898        1       5 -0.194624  0.211975
8      10.145674        1       6  0.221690 -0.038337
7       7.723774        1       7  0.180767  0.040339
5       7.491034        1       8 -0.186793  0.228257
6       7.316703        1       9 -0.088755 -0.303189
0       7.198444        1      10  0.267245  0.155961, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
51    Default  3582.000000           love  3582.000000  30.0000  30.0000
20    Default  1236.000000       daughter  1236.000000  29.0000  29.0000
42    Default  1173.000000          great  1173.000000  28.0000  28.0000
96    Default  1370.000000            toy  1370.000000  27.0000  27.0000
0     Default  1342.000000             's  1342.000000  26.0000  26.0000
70    Default   752.000000        qualiti   752.000000  25.0000  25.0000
55    Default   903.000000            n't   903.000000  24.0000  24.0000
19    Default   907.000000           cute   907.000000  23.0000  23.0000
35    Default  1286.000000           good  1286.000000  22.0000  22.0000
21    Default   561.000000  daughter love   561.000000  21.0000  21.0000
98    Default  1611.000000           veri  1611.000000  20.0000  20.0000
58    Default   637.000000            old   637.000000  19.0000  19.0000
15    Default   503.000000       christma   503.000000  18.0000  18.0000
103   Default   486.000000           year   486.000000  17.0000  17.0000
87    Default   552.000000        squishi   552.000000  16.0000  16.0000
48    Default  1135.000000          littl  1135.000000  15.0000  15.0000
83    Default   518.000000          smell   518.000000  14.0000  14.0000
6     Default   555.000000           babi   555.000000  13.0000  13.0000
43    Default   419.000000          happi   419.000000  12.0000  12.0000
45    Default   605.000000           just   605.000000  11.0000  11.0000
39    Default   448.000000    granddaught   448.000000  10.0000  10.0000
84    Default   869.000000           soft   869.000000   9.0000   9.0000
68    Default   446.000000        product   446.000000   8.0000   8.0000
47    Default   951.000000           like   951.000000   7.0000   7.0000
104   Default   371.000000       year old   371.000000   6.0000   6.0000
66    Default   400.000000        present   400.000000   5.0000   5.0000
27    Default   367.000000            doe   367.000000   4.0000   4.0000
22    Default   362.000000            day   362.000000   3.0000   3.0000
33    Default   381.000000           gift   381.000000   2.0000   2.0000
10    Default   905.000000         bought   905.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
4     Topic10    10.028873            ani   201.498405  -0.3690  -5.8153
48    Topic10    11.517135          littl  1135.610465  -1.9598  -5.6770
10    Topic10     5.360635         bought   905.889911  -2.4985  -6.4417
60    Topic10     0.100661          order   158.929085  -4.7332 -10.4168
37    Topic10     0.100651      good size   162.577556  -4.7559 -10.4169
97    Topic10     0.104753            use   170.809046  -4.7654 -10.3770
100   Topic10     0.100647     veri happi   164.828665  -4.7697 -10.4170
23    Topic10     0.100650        definit   167.279580  -4.7845 -10.4169
102   Topic10     0.100648          worth   172.856781  -4.8173 -10.4169
28    Topic10     0.100647        doe n't   181.555054  -4.8664 -10.4170
32    Topic10     0.100650            fun   182.467921  -4.8714 -10.4169
50    Topic10     0.100657            lot   184.149119  -4.8805 -10.4168
2     Topic10     0.1006

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:25:37.768915. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [48]:
# #挑選特定條件(抽樣)
# samples = ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 4][:441]
# Rsamples = samples['review'].values
# Rsamples

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:25:39.748119. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [49]:
# with open('../test.csv','w',encoding='utf-8') as f :
#     f.write(str(Rsamples))

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 14:25:39.758462. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
